In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline as pl
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, LinearSVR
from sklearn.metrics import f1_score, precision_score, accuracy_score

data = pd.read_csv('Car_sales.csv')
# print(data.dtypes, data.isna().sum())
# sns.heatmap(data.isnull())
# plt.show()
# print(data.head())
data = data.drop('Latest_Launch', axis=1)
test = data.drop('Vehicle_type', axis = 1)
test_ids = data['Vehicle_type']

def cleaning_service(data):
    cat_cols = ['Manufacturer', 'Model']
    cols = ['Sales_in_thousands', '__year_resale_value', 'Price_in_thousands', 'Length', 'Engine_size',
             'Horsepower', 'Wheelbase', 'Width', 'Curb_weight', 'Fuel_capacity', 'Fuel_efficiency', 'Power_perf_factor']
    unique_man = data['Manufacturer'].unique()
    unique_model = data['Model'].unique()
    for i in cols:
        data[i].fillna(data[i].median(), inplace=True)
    for i in cat_cols:
        data[i].fillna(data[i].mode()[0])
    data['Model'] = data['Model'].map(dict(zip(unique_model, list(range(len(unique_model)))))).astype(int)
    data['Manufacturer'] = data['Manufacturer'].map(dict(zip(unique_man, list(range(len(unique_man)))))).astype(int)
    return data
data = cleaning_service(data)
test = cleaning_service(test)
def main_target(data):
    unique_vehicle = data['Vehicle_type'].unique()
    data['Vehicle_type'] = data['Vehicle_type'].map(dict(zip(unique_vehicle, list(range(len(unique_vehicle)))))).astype(int)
    return data
data = main_target(data)
print(data)
# print(test)
# sns.heatmap(data.isnull())
# plt.show()
np.random.seed(42)
data_shuffled = data.sample(frac=1)
X = data.drop('Vehicle_type', axis=1)
y = data['Vehicle_type']
train_split = round(0.8 * len(data_shuffled))
valid_split = round(train_split + 0.1 * len(data_shuffled))
X_train, y_train = X[:train_split], y[:train_split]
X_valid, y_valid = X[train_split:valid_split], y[train_split:valid_split]
X_test, y_test = X[valid_split:], y[valid_split:]

balance = np.sum(y) / len(y)
print(balance)

lsvc = LinearSVC().fit(X_train, y_train)
prediction = lsvc.predict(X_test)
print('F1_score: ' + str(f1_score(prediction, y_test)))
print('Precision_score: ' + str(precision_score(prediction, y_test)))
print('Accuracy_score: ' + str(accuracy_score(prediction, y_test)))
lsvr = LinearSVR().fit(X_train, y_train)
prediction_1 = lsvr.predict(X_test)

f_pred_1 = lsvr.predict(test)
f_pred = lsvc.predict(test)

df_lsvc = pd.DataFrame({
    'PassengerId':test_ids.values,
    'Transported':f_pred
})
df_lsvr = pd.DataFrame({
    'PassengerId':test_ids.values,
    'Transported':f_pred_1
})

df_lsvc.to_csv('submission_lsvc', index=False)
df_lsvr.to_csv('submission_lsvr', index=False)

     Manufacturer  Model  Sales_in_thousands  __year_resale_value  \
0               0      0              16.919               16.360   
1               0      1              39.384               19.875   
2               0      2              14.114               18.225   
3               0      3               8.588               29.725   
4               1      4              20.397               22.255   
..            ...    ...                 ...                  ...   
152            29    151               3.545               14.180   
153            29    152              15.245               14.180   
154            29    153              17.531               14.180   
155            29    154               3.493               14.180   
156            29    155              18.969               14.180   

     Vehicle_type  Price_in_thousands  Engine_size  Horsepower  Wheelbase  \
0               0              21.500          1.8       140.0      101.2   
1               0

<ipython-input-3-d55b7158cdce>:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[i].fillna(data[i].median(), inplace=True)
<ipython-input-3-d55b7158cdce>:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 